In [43]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
from matplotlib import pyplot as plt
import pylab
import holidays
import pickle
%matplotlib inline

C:\Users\preductor\AppData\Local\conda\conda\envs\py3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
agg_data_01r = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_3\agg_data_01.csv')
agg_data_02r = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_3\agg_data_02.csv')
agg_data_03r = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_3\agg_data_03.csv')
agg_data_04r = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_3\agg_data_04.csv')
agg_data_05r = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_3\agg_data_05.csv')
agg_data_06r = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_3\agg_data_06.csv')
agg_data_15_07r = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_3\agg_data_15-07.csv')
agg_data_15_08r = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_3\agg_data_15-08.csv')
agg_data_15_09r = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_3\agg_data_15-09.csv')
agg_data_15_10r = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_3\agg_data_15-10.csv')
agg_data_15_11r = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_3\agg_data_15-11.csv')
agg_data_15_12r = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_3\agg_data_15-12.csv')


## 1.Для каждой из шести задач прогнозирования сформируйте выборки.

### Агрегируем все данные за год

In [ ]:
list_of_aggs = [agg_data_15_07, agg_data_15_08, agg_data_15_09, agg_data_15_10, 
                agg_data_15_11, agg_data_15_12, agg_data_01, agg_data_02, agg_data_03,
                agg_data_04, agg_data_05, agg_data_06]

In [77]:
def process_agg(agg):
    rng = pd.date_range('8/1/2015 00:00', periods=agg.shape[0], freq='H')    
    return agg.set_index(rng)

In [78]:
agg_data_15_08 = process_agg(agg_data_15_08r)

In [ ]:
# agg_data_05r.rename(columns={'Unnamed: 0':'day', 'Unnamed: 1':'hours'}, inplace=True)

In [ ]:
# agg_data_15_12r.sort_values(by=['day','hours'], inplace=True)

In [ ]:
agg_data = pd.concat(list_of_aggs, axis=0)

In [ ]:
with open('agg_data.pkl', 'wb') as f:
    pickle.dump(agg_data, f)

In [ ]:
agg_data.shape

### Оставляем только данные из 102 самых активных районов

In [ ]:
with open('list_of_more_5.pkl','rb') as f:
    list_of_more_5 = pickle.load(f)

In [ ]:
agg_data = agg_data.loc[:, list_of_more_5.astype(str)]
agg_data.head()

## ----------------------------------------------------------------------------------------------

In [ ]:
with open('agg_data.pkl', 'rb') as f:
    agg_data = pickle.load(f)

### Всевозможные значения T легко определить с помощью функции stack()

In [ ]:
# raw_data = agg_data.stack().reset_index(level=[1])
# data = raw_data.copy()

In [ ]:
# models_count = 6
# last_index = agg_data.tail(1).index.shift(-models_count, freq='H').values[0]
# data.drop(index=data[data.index>last_index].index, inplace=True)
# data.columns=['region', 'trips']
# data.head()

### Обозначим все targets для регрессий

In [ ]:
# for i in range(1, models_count+1):
#     try:
#         data['target_{}'.format(i)] = agg_data.iloc[i:-models_count+i, :].stack().values
#     except:
#         data['target_{}'.format(i)] = agg_data.iloc[i:, :].stack().values

In [ ]:
data.shape

### Определим признаки для регрессий для начала из прошлой недели

In [ ]:
# data['holiday'] = [1 if (x in holidays.US()) else 0 for x in data.index]

In [ ]:
# bh = pd.date_range(start=agg_data.index[0], end=agg_data.index[-1], freq='BH')
# data['is_business_hour'] = [i in bh for i in data.index]
# data['is_business_hour'] = data['is_business_hour'].astype(int)

In [ ]:
# data['hour'] = data.index.hour
# data['month'] = data.index.month
# data['year'] = data.index.year
# data['date'] = data.index.day

In [ ]:
# data['weekday'] = [x.weekday() for x in data.index]

In [ ]:
# num_of_harmonics = 15
# for K in range(1,num_of_harmonics+1):
#     data['sin_'+str(K)] = [np.sin((x//102+1)*(2.*np.pi*K)/168) for x in range(data.shape[0])]
#     data['cos_'+str(K)] = [np.cos((x//102+1)*(2.*np.pi*K)/168) for x in range(data.shape[0])]

### И новые признаки

In [ ]:
# X = data.loc[agg_data_15_08.index[0]:,:]

In [ ]:
# Kd = 2
# for i in range(1,Kd+1): 
#     X['y-{}'.format(i*24)] = data.loc[:,'trips'].shift(+i*24).loc[agg_data_15_08.index[0]:]

In [ ]:
# K = 6
# for i in range(1,K+1):   
#     X['y-{}'.format(i)] = data.loc[:,'trips'].shift(+i).loc[agg_data_15_08.index[0]:]

## -------------------------------------------------------------------------------

In [ ]:
# with open('X.pkl', 'wb') as f:
#     pickle.dump(X, f)

In [ ]:
# for i in [12, 24, 24*7, 24*30]:
#     X['sum_{}'.format(i)] = agg_data.rolling(i).sum().loc[agg_data_15_08.index[0]:,:].iloc[:-6].stack().values

In [ ]:
# with open('X.pkl', 'rb') as f:
#     X = pickle.load(f)

In [ ]:
# sub_1 = X.loc[:,['holiday','is_business_hour','hour', 'year', 'weekday']].astype('category')
# sub_1 = pd.get_dummies(sub_1)
# sub_2 = X.loc[:,['region', 'month', 'date']].astype('category')
# sub_2 = sub_2.apply(LabelEncoder().fit_transform)
# # X_targets = X.loc[:,targets]
# X.drop(columns=['region','holiday','is_business_hour','hour', 'month',
#        'year', 'date', 'weekday'], inplace=True)
# sub_3 = pd.DataFrame(index=X.index, columns=X.columns, data=StandardScaler().fit_transform(X))
# X = pd.concat([sub_3,sub_1,sub_2],axis=1)

## 2. Разбейте каждую из шести выборок на три части

In [ ]:
%%time
clf_6 = result(X)

In [ ]:
train_X = X.loc[:agg_data_04.index[-1],:]
test_X = X.loc[agg_data_05.index[0]:agg_data_06.index[0],:]
hold_out = X.loc[agg_data_06.index[0]:,:]

## 3.Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016

### Здесь предыдущая обработка данных в функциях, чтоб удобно было экспериментировать

In [ ]:
def processing_1(raw_data):
    data = raw_data.copy()
    # ----------------------------------------------
    last_index = agg_data.tail(1).index.shift(-6, freq='H').values[0]
    data.drop(index=data[data.index>last_index].index, inplace=True)
    data.columns=['region', 'trips']
    # -------------------------------------------------    
    for i in range(1, models_count+1):
        try:
            data['target_{}'.format(i)] = agg_data.iloc[i:-models_count+i, :].stack().values
        except:
            data['target_{}'.format(i)] = agg_data.iloc[i:, :].stack().values
    # --------------------------------------------------    
    data['holiday'] = [1 if (x in holidays.US()) else 0 for x in data.index]    
    bh = pd.date_range(start=agg_data.index[0], end=agg_data.index[-1], freq='BH')
    data['is_business_hour'] = [i in bh for i in data.index]
    data['is_business_hour'] = data['is_business_hour'].astype(int)
    # ---------------------------------------------------
    data['hour'] = data.index.hour
    data['month'] = data.index.month
    data['year'] = data.index.year
    data['date'] = data.index.day    
    data['weekday'] = [x.weekday() for x in data.index]
    # ----------------------------------------------------
    num_of_harmonics = 15
    for K in range(1,num_of_harmonics+1):
        data['sin_'+str(K)] = [np.sin((x//102+1)*(2.*np.pi*K)/168) for x in range(data.shape[0])]
        data['cos_'+str(K)] = [np.cos((x//102+1)*(2.*np.pi*K)/168) for x in range(data.shape[0])]
    # ----------------------------------------------------    
    return data

In [ ]:
def processing_2(data):
    X = data.loc[agg_data_15_08.index[0]:,:]
    
    Kd = 7
    for i in range(1,Kd+1): 
        X['y-{}'.format(i*24)] = data.loc[:,'trips'].shift(+i*24).loc[agg_data_15_08.index[0]:]
        
    K = 12
    for i in range(1,K+1):   
        X['y-{}'.format(i)] = data.loc[:,'trips'].shift(+i).loc[agg_data_15_08.index[0]:]
        
    for i in [12, 24, 24*7, 24*30]:
        X['sum_{}'.format(i)] = agg_data.rolling(i).sum().loc[agg_data_15_08.index[0]:,
                                                              :].iloc[:-6].stack().values

    targets = ['target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6']
    
    sub_1 = X.loc[:,['holiday','is_business_hour','hour',
                     'year', 'weekday']].astype('category')
    sub_1 = pd.get_dummies(sub_1)
    sub_2 = X.loc[:,['region', 'month', 'date']].astype('category')
    sub_2 = sub_2.apply(LabelEncoder().fit_transform)
    X_targets = X.loc[:,targets]
    X.drop(columns=['region', 'month', 'date', 'holiday','is_business_hour','hour',
                     'year', 'weekday']+targets, inplace=True)
    sub_3 = pd.DataFrame(index=X.index, columns=X.columns, data=StandardScaler().fit_transform(X))
    X = pd.concat([sub_3,sub_1,sub_2,X_targets],axis=1)
       
    return X

In [45]:
warnings.simplefilter('ignore')
data = processing_1(raw_data)
X = processing_2(data)
targets = ['target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6']

### Модель сначала настраивал, потом стал работать уже с одним набором параметров

In [147]:
def result(target_name):    
    param_grid = {
        'max_depth':[3],
        'learning_rate': [0.05],
        'n_estimators': [50],
        'objective' : ['reg:linear'],
        'colsample_bytree' : [0.8],
        'subsample' : [0.7],
        'reg_alpha' : [1.2],
        'reg_lambda' : [1.2],
        'silent':[True]
        }
    fit_params = {
                "eval_set" : [(test_X.drop(columns=targets),test_X[target_name])],
                'verbose': False
                }
    
#     gscv = GridSearchCV(XGBRegressor(), param_grid, scoring='neg_mean_absolute_error', fit_params=fit_params,
#         cv=2, verbose=True)
#     gscv.fit(train_X.drop(columns=targets),train_X[target_name])
#     clf = gscv.best_estimator_

    clf = XGBRegressor(max_depth=3,
        learning_rate=0.05,
        n_estimators=50,
        objective='reg:linear',
        colsample_bytree=0.8,
        subsample=0.7,
        reg_alpha=1.2,
        reg_lambda=1.2,
        silent=True)
    
    clf.fit(train_X.drop(columns=targets), train_X[target_name], 
            eval_set=[(test_X.drop(columns=targets),test_X[target_name])],verbose=False)
    

    return clf

## 4. Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; 

In [148]:
train_X = X.loc[:agg_data_04.index[-1],:]
test_X = X.loc[agg_data_05.index[0]:,:]
test_X.drop(index=X.loc[agg_data_05.index[-7]:].index, inplace=True)
hold_out = X.loc[agg_data_05.index[-7]:]

indexes = X.loc[agg_data_05.index[-1]:,:].index
    
temp = pd.DataFrame(index=indexes, columns=['region'])

temp.region = data.region.loc[indexes.unique()].values

for i in targets:
    temp[i] = X.loc[indexes.unique(),i].values
    
for i in targets:
    clf = result(i)
    res = [int(x) for x in clf.predict(X.loc[indexes.unique(),:].drop(columns=targets))]
    temp[i+'_pred'] = res


In [155]:
temp.head(3)

,region,target_1,target_2,target_3,target_4,target_5,target_6,target_1_pred,target_2_pred,target_3_pred,target_4_pred,target_5_pred,target_6_pred
2016-05-31 23:00:00,1075,27,14,5,2,1,7,37,31,38,44,49,54
2016-05-31 23:00:00,1076,30,21,25,5,8,31,80,81,58,62,59,79
2016-05-31 23:00:00,1077,19,6,4,3,3,17,52,55,40,48,59,69


In [319]:
ftemp = temp.drop(columns=targets, axis=1)

ftemp.rename(columns={'target_1_pred':'1',
                     'target_2_pred':'2',
                     'target_3_pred':'3',
                     'target_4_pred':'4',
                     'target_5_pred':'5',
                     'target_6_pred':'6'}, inplace=True)

In [320]:
final = ftemp.copy()
final['time'] = final.index
final.sort_values(by=['region','time'], inplace=True)
# final.set_index(['region','time'], inplace=True)
final.head(3)

,region,1,2,3,4,5,6,time
2016-05-31 23:00:00,1075,37,31,38,44,49,54,2016-05-31 23:00:00
2016-06-01 00:00:00,1075,33,30,38,46,49,56,2016-06-01 00:00:00
2016-06-01 01:00:00,1075,27,26,37,46,53,56,2016-06-01 01:00:00


In [321]:
final = final.astype(str)
final.set_index(['region','time'], inplace=True)

final = final.stack().reset_index(level=[2])
final.columns = ['target', 'y']
final.head(3)

target   y
region time                          
1075   2016-05-31 23:00:00      1  37
       2016-05-31 23:00:00      2  31
       2016-05-31 23:00:00      3  38

In [346]:
%%time
results = ["id,y"]
for i in final.iterrows():
        idx, columns = i[0], i[1]
        iD = idx[0]

        date = idx[1]
        year = date[:4]
        month = date[5:7]
        day = date[8:10]
        hour = date[11:13]
        if (hour[0]=='0'):
            hour=hour[1]
        
        target = columns.target
        y = columns.y
        results.append(f'{iD}_{year}-{month}-{day}_{hour}_{target},{y}')

Wall time: 44 s


In [246]:
# solution.to_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_5\solution.csv', sep=',', index=False)


In [347]:
with open(r'C:\Users\preductor\Documents\MachineLearning\6_course\Time series analysis\week_5\solution.csv',
          'w') as f:
    f.write('\n'.join(results))

In [345]:
results[2]

'1075_2016-05-31_23_2,31'